In [7]:
# Importing Libraries 
import numpy as np
import tensorflow as tf
from keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv
from PIL import Image, ImageFont, ImageDraw

In [8]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    h = image.shape[0]
    w = image.shape[1]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    
    # return the resized image
    return resized

In [9]:
def pre_process_image(face_crop,face_gray,size=100):
    w,h = face_crop.shape[1],face_crop.shape[0]
    new_image = np.zeros((size,size,3))
    new_gray_image = np.zeros((48,48))
    if w>h:
        image = image_resize(face_crop,width=size)
        image_gray = image_resize(face_gray,width=48)
        height_offset = int((new_image.shape[0] - image.shape[0])/2)
        new_image[height_offset:int(image.shape[0]+height_offset), :image.shape[1]] = image
    elif h>w:
        image = image_resize(face_crop,height=size)
        image_gray = image_resize(face_gray,height=48)
        width_offset = int((new_image.shape[1] - image.shape[1])/2)
        new_image[:image.shape[0], width_offset:int(image.shape[1] + width_offset)] = image
    else:
        image = image_resize(face_crop,width=size,height=size)
        image_gray = image_resize(face_gray,width=48,height=48)
        new_image[:image.shape[0], :image.shape[1]] = image
    new_gray_image[:image_gray.shape[0], :image_gray.shape[1]] = image_gray
    new_gray_image = np.expand_dims(new_gray_image,axis=2)
    cv2.imshow("",new_image)
    face_crop = new_image.astype("float") / 255.0
    face_gray = new_gray_image.astype("float")/255.0
    return face_crop,face_gray

In [10]:

bmi_net = load_model("best_model_vgg_complex_100.h5")
emotion_classifier = load_model("sentiment_standard.h5")
class_map = {0: 'angry',1: 'disgust',2: 'fear',3: 'happy',4: 'neutral',5: 'sad',6: 'surprise'}
print("[INFO] starting video stream...")
webcam = cv2.VideoCapture(0)
font = ImageFont.truetype("../data/Uni Sans Heavy.otf", 12)
# loop over the frames from the video stream
while webcam.isOpened():
    status, frame = webcam.read()
    face, confidence = cv.detect_face(frame)
    new_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = []
    gray_faces = []
    bboxes = []
    preds = []
    for idx, f in enumerate(face):
        print(f)
        face_crop = []
        face_gray = []
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        cv2.rectangle(frame, (startX,startY), (endX,endY), (6,214,160), 1)
        startX = startX - 100
        startY = startY -100
        endX = endX + 100
        endY = endY + 100
        startX = 0 if startX<1 else startX
        startY = 0 if startY<1 else startY
        if endX - startX < 50:
            continue 
        face_crop = np.copy(frame[startY:endY,startX:endX])
        face_gray = np.copy(gray[startY:endY,startX:endX])
        face_crop,face_gray = pre_process_image(face_crop,face_gray,size=100)
        
        faces.append(face_crop)
        gray_faces.append(face_gray)
        Y = f[1] - 10 if f[1] - 10 > 10 else f[1] + 10
        bboxes.append((f[0],Y))
    im = Image.fromarray(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    if len(faces)>0:
        faces = np.array(faces, dtype="float32")
        gray_faces = np.array(gray_faces, dtype="float32")
        conf = bmi_net.predict(faces,batch_size=32)
        emotion = emotion_classifier.predict(gray_faces,batch_size=32)
        emotions = [(class_map[sub_emotion.argmax()],max(sub_emotion)) for sub_emotion in emotion]
        draw = ImageDraw.Draw(im)
        for location,pred,emotion in zip(bboxes,conf,emotions):
            conf_text = "BMI : " + str(int(pred))
            emotion_text = "Emotion: " + str(emotion[0]) + "-" + str(emotion[1])
            draw.text((location[0], location[1]-30), conf_text,(6,214,160),font=font)
            draw.text((location[0], location[1]-15), emotion_text,(6,214,160),font=font)
    frame = cv2.cvtColor(np.array(im),cv2.COLOR_RGB2BGR)
    # display output
    cv2.imshow("BMI Detection", frame)
    # im.show()
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        webcam.release()


: 

: 

In [8]:
webcam.release()
cv2.destroyAllWindows()